In [ ]:
!pip install pyspark
from pyspark.sql import *

spark= SparkSession.builder\
.master("local")\
.appName("Colab")\
.config('spark.ui.port', '4050')\
.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 52.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ee9c412588eeae379fae1196290379bd04dae092481c3eaa9497ceae532c2b77
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
from  pyspark import pandas

In [ ]:
import pandas as pd
import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/threnjen/2019-airline-delays-and-cancellations?select=full_data_flightdelay.csv")
##  alialnujaidi

##  3b1e0a628c830e978368ab04b89840b1

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Your Kaggle Key: Downloading 2019-airline-delays-and-cancellations.zip to ./2019-airline-delays-and-cancellations


100%|██████████| 754M/754M [00:05<00:00, 132MB/s]


In [ ]:
df = spark.read.option("header",True) \
     .csv("2019-airline-delays-and-cancellations/full_data_flightdelay.csv")

In [ ]:
df.show()

+-----+-----------+---------+------------+--------------+--------------+------------------+---------------+--------------------+---------------------+---------------------+-----------------------------+------------------------+------------------------+-----------------------+--------------------+---------+--------------------+--------+---------+----------------+----+----+----+----+----+
|MONTH|DAY_OF_WEEK|DEP_DEL15|DEP_TIME_BLK|DISTANCE_GROUP|SEGMENT_NUMBER|CONCURRENT_FLIGHTS|NUMBER_OF_SEATS|        CARRIER_NAME|AIRPORT_FLIGHTS_MONTH|AIRLINE_FLIGHTS_MONTH|AIRLINE_AIRPORT_FLIGHTS_MONTH|AVG_MONTHLY_PASS_AIRPORT|AVG_MONTHLY_PASS_AIRLINE|FLT_ATTENDANTS_PER_PASS|GROUND_SERV_PER_PASS|PLANE_AGE|   DEPARTING_AIRPORT|LATITUDE|LONGITUDE|PREVIOUS_AIRPORT|PRCP|SNOW|SNWD|TMAX|AWND|
+-----+-----------+---------+------------+--------------+--------------+------------------+---------------+--------------------+---------------------+---------------------+-----------------------------+------------------

In [ ]:
df = df.select('DISTANCE_GROUP','NUMBER_OF_SEATS','AIRPORT_FLIGHTS_MONTH','GROUND_SERV_PER_PASS','PRCP','SNOW','SNWD','TMAX','AWND','DEP_DEL15')

In [ ]:
df.show()

+--------------+---------------+---------------------+--------------------+----+----+----+----+----+---------+
|DISTANCE_GROUP|NUMBER_OF_SEATS|AIRPORT_FLIGHTS_MONTH|GROUND_SERV_PER_PASS|PRCP|SNOW|SNWD|TMAX|AWND|DEP_DEL15|
+--------------+---------------+---------------------+--------------------+----+----+----+----+----+---------+
|             2|            143|                13056|9.889412309998219...| 0.0| 0.0| 0.0|65.0|2.91|        0|
|             7|            191|                13056|0.000148660200942...| 0.0| 0.0| 0.0|65.0|2.91|        0|
|             7|            199|                13056|0.000148660200942...| 0.0| 0.0| 0.0|65.0|2.91|        0|
|             9|            180|                13056|0.000148660200942...| 0.0| 0.0| 0.0|65.0|2.91|        0|
|             7|            182|                13056|0.000124651073071...| 0.0| 0.0| 0.0|65.0|2.91|        0|
|             3|            180|                13056|7.134694872433899...| 0.0| 0.0| 0.0|65.0|2.91|        0|
|

In [ ]:
df.printSchema()

root
 |-- DISTANCE_GROUP: string (nullable = true)
 |-- NUMBER_OF_SEATS: string (nullable = true)
 |-- AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- GROUND_SERV_PER_PASS: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- SNOW: string (nullable = true)
 |-- SNWD: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- AWND: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="DEPARTING_AIRPORT", outputCol="AirportIndex") 
indexed = indexer.fit(df).transform(df) 

In [ ]:
df.show()

In [ ]:
df = indexed.drop('DEPARTING_AIRPORT')

In [ ]:
df = df.withColumn("DISTANCE_GROUP",col("DISTANCE_GROUP").cast("double")).dropna("any")
df = df.withColumn("NUMBER_OF_SEATS",col("NUMBER_OF_SEATS").cast("double")).dropna("any")
df = df.withColumn("AIRPORT_FLIGHTS_MONTH",col("AIRPORT_FLIGHTS_MONTH").cast("double")).dropna("any")
df = df.withColumn("GROUND_SERV_PER_PASS",col("GROUND_SERV_PER_PASS").cast("double")).dropna("any")
df = df.withColumn("PRCP",col("PRCP").cast("double")).dropna("any")
df = df.withColumn("SNOW",col("SNOW").cast("double")).dropna("any")
df = df.withColumn("SNWD",col("SNWD").cast("double")).dropna("any")
df = df.withColumn("TMAX",col("TMAX").cast("double")).dropna("any")
df = df.withColumn("AWND",col("AWND").cast("double")).dropna("any")
df = df.withColumn("DEP_DEL15",col("DEP_DEL15").cast("int")).dropna("any")

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
input_cols = ['DISTANCE_GROUP','NUMBER_OF_SEATS','AIRPORT_FLIGHTS_MONTH','GROUND_SERV_PER_PASS','PRCP','SNOW','SNWD','TMAX','AWND']
assembler = VectorAssembler(inputCols=input_cols, outputCol='features')
assembler_temp = assembler.transform(df)

In [ ]:
assembler_temp.show()

In [ ]:
new_df = assembler_temp.drop('DISTANCE_GROUP','NUMBER_OF_SEATS','AIRPORT_FLIGHTS_MONTH','GROUND_SERV_PER_PASS','PRCP','SNOW','SNWD','TMAX','AWND')

In [ ]:
new_df.show()

In [ ]:
(trainData, testData) = new_df.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
dt_model = DecisionTreeClassifier(labelCol="DEP_DEL15", featuresCol="features", maxBins=100)

In [ ]:
model = dt_model.fit(trainData)

In [ ]:
prediction = model.transform(testData)
prediction.show()

+---------+--------------------+--------------------+--------------------+----------+
|DEP_DEL15|            features|       rawPrediction|         probability|prediction|
+---------+--------------------+--------------------+--------------------+----------+
|        0|(9,[0,1,2,3],[1.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[1.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[0.82943424628157...|       0.0|
|        0|(9,[0,1,2,3],[5.0...|[3012934.0,619583.0]|[

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="DEP_DEL15",predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Accuracy For the Second Classification model: ",accuracy)
print("Test Error For the Second Classification model = %g" %(1.0 - accuracy ))

Accuracy For the Second Classification model:  0.811015205832419
Test Error For the Second Classification model = 0.188985
